# Дашборд для Яндекс.Дзен

## Содержание

[Оглавление](#intro)
1. [Описание проекта, цели и задачи](#intro1)
2. [Описание данных](#intro2)
3. [Макет дашборда](#intro3)
4. [Предоработка данных](#intro4)
5. [Ссылка на публикацию дашборда:](#intro5)
8. [Ссылка на презентацию](#intro6)

## [Описание проекта, цели и задачи](#intro1)

На регулярные вопросы менеждеров решено построить дашборд, который будет отвечать на следующие вопросы: 
* Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
* Как много карточек генерируют источники с разными темами?
* Как соотносятся темы карточек и темы источников?

**Подготовка:** Нужно подробно обсудить с менеджерами состав дашборда, его внешний вид и набор отображаемых данных. После этого пообщаться с администраторами БД и выяснить, куда и как собираются нужные данные. С ними же нужно решить, где хранить агрегирующие таблицы. В финале приступить к разработке пайплайна и дашборда.

**Краткое ТЗ:**

Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;

* Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
* Кто будет основным пользователем дашборда: менеджеры по анализу контента;

Состав данных для дашборда:
* История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
* Разбивка событий по темам источников;
* Таблица соответствия тем источников темам карточек;

По каким параметрам данные должны группироваться:
* Дата и время;
* Тема карточки;
* Тема источника;
* Возрастная группа;

Характер данных:
* История событий по темам карточек — абсолютные величины с разбивкой по минутам;
* Разбивка событий по темам источников — относительные величины (% событий);
* Соответствия тем источников темам карточек - абсолютные величины;

Важность: все графики имеют равную важность;

## [Описание данных](#intro2)

Источники данных для дашборда: дата-инженеры обещали подготовить для вас агрегирующую таблицу dash_visits. 
    
Cтруктура данных:
* record_id — первичный ключ,
* item_topic — тема карточки,
* source_topic — тема источника,
* age_segment — возрастной сегмент,
* dt — дата и время,
* visits — количество событий.

Таблица хранится в специально подготовленной для вас базе данных zen;
Частота обновления данных: один раз в сутки, в полночь по UTC;
Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):

## [Макет дашборда](#intro3)

<a href="https://ibb.co/fY5g19g"><img src="https://i.ibb.co/JR41Bq1/Maket-dashboard.png" alt="Maket-dashboard" border="0"></a>

In [1]:
# установим библиотеку

#!pip install psycopg2

In [2]:
# Импортируем необходимые библиотеки

import pandas as pd
from sqlalchemy import create_engine
import sys
import getopt
import psycopg2

In [3]:
# Задаем параметры подключения к серверу

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [4]:
# Формируем sql-запрос

query = ''' SELECT *             
            FROM dash_visits
 
        '''
 
# Выполняем запрос и сохраняем результат
# выполнения в DataFrame.
# Sqlalchemy автоматически установит названия колонок
# такими же, как у таблицы в БД. Нам останется только
# указать индексную колонку с помощью index_col.
dash_visits = pd.io.sql.read_sql(query, con = engine)
 
# Сделаем предварительный обзор полученных данных:
print(dash_visits.head())

   record_id item_topic source_topic age_segment                  dt  visits
0    1040597     Деньги         Авто       18-25 2019-09-24 18:32:00       3
1    1040598     Деньги         Авто       18-25 2019-09-24 18:35:00       1
2    1040599     Деньги         Авто       18-25 2019-09-24 18:54:00       4
3    1040600     Деньги         Авто       18-25 2019-09-24 18:55:00      17
4    1040601     Деньги         Авто       18-25 2019-09-24 18:56:00      27


## [Предоработка данных](#intro4)

In [5]:
# Выведим основную информацию о датафрейме с помощью метода info()
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [6]:
# Выводим первые 5 строчек датафрейма 'data' на экран
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [7]:
# Найдем все пропуски в таблице и посчитаем их количество:
dash_visits.isna().sum()

record_id       0
item_topic      0
source_topic    0
age_segment     0
dt              0
visits          0
dtype: int64

In [8]:
# Поиск полных дубликатов
dash_visits.duplicated().sum()

0

In [9]:
# Список уникальных значений в тема карточки
dash_visits['item_topic'].unique()

array(['Деньги', 'Дети', 'Женская психология', 'Женщины', 'Здоровье',
       'Знаменитости', 'Интересные факты', 'Искусство', 'История',
       'Красота', 'Культура', 'Наука', 'Общество', 'Отношения',
       'Подборки', 'Полезные советы', 'Психология', 'Путешествия',
       'Рассказы', 'Россия', 'Семья', 'Скандалы', 'Туризм', 'Шоу', 'Юмор'],
      dtype=object)

In [10]:
# Список уникальных значений в тема источника
dash_visits['source_topic'].unique()

array(['Авто', 'Деньги', 'Дети', 'Еда', 'Здоровье', 'Знаменитости',
       'Интерьеры', 'Искусство', 'История', 'Кино', 'Музыка', 'Одежда',
       'Полезные советы', 'Политика', 'Психология', 'Путешествия',
       'Ремонт', 'Россия', 'Сад и дача', 'Сделай сам',
       'Семейные отношения', 'Семья', 'Спорт', 'Строительство',
       'Технологии', 'Финансы'], dtype=object)

In [11]:
# Список уникальных значений в возрастном сегменте
dash_visits['age_segment'].unique()

array(['18-25', '26-30', '31-35', '36-40', '41-45', '45+'], dtype=object)

In [12]:
# Смотрим описание величин для всех числовых типов данных: количество, среднее арифметическое, стандартное отклонение,
# минимум, Q1, Q2 - медиана, Q3, максимум 
dash_visits.describe().round(2)

,record_id,visits
count,30745.00,30745.00
mean,1055969.00,10.09
std,8875.46,19.73
min,1040597.00,1.00
25%,1048283.00,1.00
50%,1055969.00,3.00
75%,1063655.00,10.00
max,1071341.00,371.00


In [13]:
# Cмотрим описание статистику к нечисловым столбцам (категориальному типу данных): 
# количество, уникальные, самые частые, их частота
dash_visits.describe(include=['O'])

,item_topic,source_topic,age_segment
count,30745,30745,30745
unique,25,26,6
top,Отношения,Семейные отношения,18-25
freq,1536,1822,7056


In [14]:
# Выгрузим полученные данные в файл dash_visits.csv

dash_visits.to_csv('dash_visits.csv', index=False)

## [Ссылка на публикацию дашборда:](#intro5)

https://public.tableau.com/app/profile/ekaterina.drevetnyak/viz/__16815508960310/Dashboard4

## [Ссылка на презентацию](#intro6)

https://disk.yandex.ru/i/4VVFfYplN5M-qQ